# Haystack: Monitoring LLMs for Agents (WIP)

This guide will navigate you through utilizing ArgillaCallback to monitor the LLMS of your Haystack agents and seamlessly log the results into your Argilla Server. Haystack provides powerful tools for constructing and overseeing comprehensive pipelines for LLM production, enabling smooth collaboration with Conversational or RAG models, among others. The seamless integration with Argilla further simplifies the process of monitoring these pipelines. As a result, you gain the flexibility to effortlessly seek human feedback for assessing and optimizing your model's performance.

In this tutorial, we will see two examples where we can benefit from ArgillaCallback, the first one being a simple example of logging into Argilla while the second one being a more advanced one where we will employ some tools for our agent.

## Conversational Agent

To use ArgillaCallback within your Haystack workflow, you first need to create an agent. For this tutorial, we will use a simple conversational agent that will be able to answer questions. We will also use GPT3.5 from OpenAI as our LLM. For this, you will need a valid API key from OpenAI. You can have more info and get one via [this link](https://openai.com/blog/openai-api).

After you get your API key, let us import the key.

In [ ]:
import os
from getpass import getpass

openai_api_key = os.getenv("OPENAI_API_KEY", None) or getpass("Enter OpenAI API key:")

The code snippet below will create a simple conversational agent using Haystack `PromptNode`. 

In [ ]:
from haystack.nodes import PromptNode
from haystack.agents.memory import ConversationSummaryMemory
from haystack.agents.conversational import ConversationalAgent

prompt_node = PromptNode(
    model_name_or_path="gpt-3.5-turbo-instruct", api_key=openai_api_key, max_length=256, stop_words=["Human"]
)
summary_memory = ConversationSummaryMemory(prompt_node)
conversational_agent = ConversationalAgent(prompt_node=prompt_node, memory=summary_memory)

Now that we have an up and running agent, we can employ the `ArgillaCallback`. To initialize the callback, you will need to provide the agent we have created and also the name of the dataset in the Argilla Server, where it will log in the results. Note that, if you do not have a dataset with the given name, it will create one for you. The created dataset will have the fields of `prompt` and `response` while also a `RatingQuestion` with values from 1 to 5.

To properly connect to your server, you will also need to provide the api_key and the api_url of your server. If not provided, it will use the default values. Additionally, you have the opportunity to provide the workspace that you will specifically work within. In case it is not provided, again it will use the default workspace.

In [ ]:
from haystack_argilla.base import ArgillaCallback

api_key = "argilla.apikey"
api_url = "http://localhost:6900/"
dataset_name = "conversational_ai"

ArgillaCallback(agent=conversational_agent, dataset_name=dataset_name, api_url=api_url, api_key=api_key)

With the agent defined and callback initialized, we are now ready to monitor our agent and log into Argilla. Let us run the agent.

In [14]:
conversational_agent.run("Tell me three most interesting things about Istanbul, Turkey")


Agent conversational-agent-without-tools started with {'query': 'Tell me three most interesting things about Istanbul, Turkey', 'params': None}
 Istanbul is a city with a rich history and culture, here are three interesting things about it:

1. The Grand Bazaar in Istanbul is one of the oldest and largest covered markets in the world, dating back to the 15th century. It is a bustling hub of trade and commerce, with over 4,000 shops selling everything from spices to carpets.

2. Istanbul is home to the largest palace in Europe, the Topkapi Palace. It was the residence of the Ottoman sultans for nearly 400 years and is now a popular tourist attraction.

3. Istanbul has a unique and delicious cuisine that combines elements from both European and Middle Eastern cultures. Some popular dishes include kebabs, baklava, and Turkish delight.

Output()

[01/05/24 14:27:55] INFO     INFO:haystack_argilla.base:Records have been updated to Argilla            ]8;id=746317;file:///Users/kursat/argilla-haystack/haystack_argilla/base.py\base.py]8;;\:]8;id=416370;file:///Users/kursat/argilla-haystack/haystack_argilla/base.py#214\214]8;;\

{'query': 'Tell me three most interesting things about Istanbul, Turkey',
 'answers': [<Answer {'answer': 'Istanbul is a city with a rich history and culture, here are three interesting things about it:\n\n1. The Grand Bazaar in Istanbul is one of the oldest and largest covered markets in the world, dating back to the 15th century. It is a bustling hub of trade and commerce, with over 4,000 shops selling everything from spices to carpets.\n\n2. Istanbul is home to the largest palace in Europe, the Topkapi Palace. It was the residence of the Ottoman sultans for nearly 400 years and is now a popular tourist attraction.\n\n3. Istanbul has a unique and delicious cuisine that combines elements from both European and Middle Eastern cultures. Some popular dishes include kebabs, baklava, and Turkish delight.', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': None, 'meta': {}}>],
 'transcript': ' Istanbul is a city wi

As seen, the agent responded to the query and gave us the response. The records have also been updated to the Argilla Server. Let us check it on the Argilla Server.

![Argilla Dataset](argilla-dataset.png)

## Use Agent to Search Your Documents

If you would like to employ your agent in a more advanced workflow, you can turn it to a decision-maker, where it determines the best course of action in a given situation. Haystac agents makes use of various Tools or Memory components to achieve a good performance where the task requires versatile LLM skills and each specific subtask needs to be handled differently. This tutorial is taken from the [Haystack documentation](https://haystack.deepset.ai/tutorials/25_customizing_agent) and you can find more tutorials and examples there.

For this tutorial, we will again use GPT3.5 from OpenAI as our LLM. For this, you will need a valid API key from OpenAI. You can have more info and get one via [this link](https://openai.com/blog/openai-api).

After you get your API key, let us import the key.

In [ ]:
import os
from getpass import getpass

openai_api_key = os.getenv("OPENAI_API_KEY", None) or getpass("Enter OpenAI API key:")

Let us import necessary libraries.

In [34]:
from haystack_argilla.base import ArgillaCallback
import argilla as rg
from datasets import load_dataset
from haystack.document_stores import InMemoryDocumentStore
from haystack.nodes import PromptNode, PromptTemplate, AnswerParser, BM25Retriever
from haystack.pipelines import Pipeline

from haystack.agents import Tool
from haystack.agents.memory import ConversationSummaryMemory
from haystack.agents import AgentStep, Agent
from haystack.agents.base import Agent, ToolsManager

Let us load the data.

In [17]:
dataset = load_dataset("bilgeyucel/seven-wonders", split="train")

Then, let us create the generative pipeline that our agent will be using.

In [ ]:
document_store = InMemoryDocumentStore(use_bm25=True)
document_store.write_documents(dataset)
retriever = BM25Retriever(document_store=document_store)
prompt0 = "Please use a maximum of 50 tokens. Question: {query}\nDocuments: {join(documents)}\nAnswer:"
prompt_template = PromptTemplate(
    prompt=prompt0,
    output_parser=AnswerParser(),
)
prompt_node = PromptNode(
    model_name_or_path="gpt-3.5-turbo-instruct", api_key=openai_api_key, default_prompt_template=prompt_template
)
generative_pipeline = Pipeline()
generative_pipeline.add_node(component=retriever, name="Retriever", inputs=["Query"])
generative_pipeline.add_node(component=prompt_node, name="Prompt", inputs=["Retriever"])

With the pipeline defined, let us create the tool. 

In [ ]:
search_tool = Tool(
    name="Search_the_documents_tool",
    pipeline_or_node=generative_pipeline,
    description="useful for when you need to answer questions about the seven wonders of the world",
    output_variable="answers",
)
agent_prompt_node = PromptNode(
    "gpt-3.5-turbo",
    api_key=openai_api_key,
    max_length=256,
    stop_words=["Observation:"],
    model_kwargs={"temperature": 0.5},
)
memory_prompt_node = PromptNode(
    model_name_or_path="philschmid/bart-large-cnn-samsum", max_length=256, model_kwargs={"task_name":"text2text-generation"} 
)
memory = ConversationSummaryMemory(
    prompt_node=memory_prompt_node, prompt_template="{chat_transcript}"
)

We will need a prompt which will be used to query the tool by the agent. It should be sufficiently detailed to obtain a good response from the agent.

In [20]:
agent_prompt = """
In the following conversation, a human user interacts with an AI Agent. The human user poses questions, and the AI Agent goes through several steps to provide well-informed answers.
The AI Agent must use the available tools to find the up-to-date information. The final answer to the question should be truthfully based solely on the output of the tools. The AI Agent should ignore its knowledge when answering the questions.
The AI Agent has access to these tools:
{tool_names_with_descriptions}

The following is the previous conversation between a human and The AI Agent:
{memory}

AI Agent responses must start with one of the following:

Thought: [the AI Agent's reasoning process]
Tool: [tool names] (on a new line) Tool Input: [input as a question for the selected tool WITHOUT quotation marks and on a new line] (These must always be provided together and on separate lines.)
Observation: [tool's result]
Final Answer: [final answer to the human user's question]

When selecting a tool, the AI Agent must provide both the "Tool:" and "Tool Input:" pair in the same response, but on separate lines.

The AI Agent should not ask the human user for additional information, clarification, or context.
If the AI Agent cannot find a specific answer, it should accept the first word of the answer it found as the answer to the query.

Question: {query}
Thought:
{transcript}
"""

As the last step before creating the agent, we need a resolver function that will handle the prompt given.

In [21]:
def resolver_function(query, agent, agent_step):
    return {
        "query": query,
        "tool_names_with_descriptions": agent.tm.get_tool_names_with_descriptions(),
        "transcript": agent_step.transcript,
        "memory": agent.memory.load(),
    }

We are now ready to define the agent with the given pipeline and prompt above.

In [29]:
conversational_agent = Agent(
    agent_prompt_node,
    prompt_template=agent_prompt,
    prompt_parameters_resolver=resolver_function,
    memory=memory,
    tools_manager=ToolsManager([search_tool]),
)

After getting the agent ready, we will be initializing the `ArgillaCallback` as we did in the previous example. With the same credentiials, let us initialize the callback.

In [ ]:
api_key = "argilla.apikey"
api_url = "http://localhost:6900/"
dataset_name = "search_the_documents"

ArgillaCallback(agent=conversational_agent, dataset_name=dataset_name, api_url=api_url, api_key=api_key)

In [33]:
conversational_agent.run("Where is the temple of Artemis?")


Agent custom-at-query-time started with {'query': 'Where is the temple of Artemis?', 'params': None}
The temple of Artemis is one of the seven wonders of the ancient world. To find its location, I will use the Search_the_documents_tool. 
Tool: Search_the_documents_tool
Tool Input: "temple of Artemis location"
Observation: The Temple of Artemis was located near the ancient city of Ephesus, about 75 kilometres (47 mi) south from the modern port city of İzmir, in Turkey.
Thought: The Search_the_documents_tool provided the information that the Temple of Artemis was located near the ancient city of Ephesus, about 75 kilometers south from the modern port city of İzmir, in Turkey.
Final Answer: The temple of Artemis is located in Turkey, near the ancient city of Ephesus.

Output()

[01/05/24 15:08:33] INFO     INFO:haystack_argilla.base:Records have been updated to Argilla            ]8;id=165015;file:///Users/kursat/argilla-haystack/haystack_argilla/base.py\base.py]8;;\:]8;id=798323;file:///Users/kursat/argilla-haystack/haystack_argilla/base.py#214\214]8;;\

{'query': 'Where is the temple of Artemis?',
 'answers': [<Answer {'answer': 'The temple of Artemis is located in Turkey, near the ancient city of Ephesus.', 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': None, 'meta': {}}>],
 'transcript': 'The temple of Artemis is one of the seven wonders of the ancient world. To find its location, I will use the Search_the_documents_tool. \nTool: Search_the_documents_tool\nTool Input: "temple of Artemis location"\nObservation: The Temple of Artemis was located near the ancient city of Ephesus, about 75 kilometres (47 mi) south from the modern port city of İzmir, in Turkey.\nThought:The Search_the_documents_tool provided the information that the Temple of Artemis was located near the ancient city of Ephesus, about 75 kilometers south from the modern port city of İzmir, in Turkey.\nFinal Answer: The temple of Artemis is located in Turkey, near the ancient city of Ephesus.'

As well as giving the response to the user, the agent has also logged the results into the Argilla Server. Let us check it on the Argilla Server.

![Argilla Dataset 2](argilla-dataset-2.png)

`ArgillaCallback` has also logged the name of the tool used and the thought process of the agent into the record as `metadata`. Let us see the metadata of the record.

In [40]:
dataset = rg.FeedbackDataset.from_argilla(dataset_name)
dataset[0].metadata

{'tool_output': 'The Search_the_documents_tool provided the information that the Temple of Artemis was located near the ancient city of Ephesus, about 75 kilometers south from the modern port city of İzmir, in Turkey.\nFinal Answer: The temple of Artemis is located in Turkey, near the ancient city of Ephesus.',
 'tool_name': 'Search_the_documents_tool'}

As seen, the tool we named as "Search_the_documents_tool" has been used by the agent and the thought process has been logged into the record. This data can be quite beneficial for further analysis and optimization of the agent according to your project.